In [ ]:
import os
import pandas as pd
import dask.dataframe as dd
from s3fs import S3FileSystem
import numpy as np
import coiled
from distributed import Client
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
## Uncomment the following cell when selecting a subset of the data

In [ ]:
# cluster = coiled.Cluster(
#     # worker_vm_types=["m6i.4xlarge"],
#     worker_vm_types=["m6i.xlarge"],
#     scheduler_vm_types=["m6i.2xlarge"],
#     name="dask-engineering-20d6aa4e-3",
#     package_sync=True, # copy local packages,
#     shutdown_on_close=False,  # reuse cluster across runs
#     show_widget=False,
#     n_workers=20,
#     use_best_zone=True,
#     account="dask-engineering",
#     backend_options={"region": "us-east-2", "spot": True, "spot_on_demand_fallback": True}
#     )
#
# client = Client(cluster)
#
#
# def subset_data():
#     print("loading data")
#     to_exclude=["string", "category", "object"]
#     ddf= dd.read_parquet("s3://prefect-dask-examples/nyc-uber-lyft/processed_files.parquet")
#     # ddf = ddf.drop(columns=["base_passenger_fare", "sales_tax", "bcf", "congestion_surcharge", "tips", "driver_pay", "dropoff_datetime"])
#     ddf = ddf.assign(accessible_vehicle = 1)
#     print("Make accessible feature")
#     ddf.accessible_vehicle = ddf.accessible_vehicle.where(ddf.on_scene_datetime.isnull(),0)  # Only applies if the vehicle is wheelchair accessible
#     ddf = ddf.assign(pickup_month = ddf.pickup_datetime.dt.month)
#     ddf = ddf.assign(pickup_dow = ddf.pickup_datetime.dt.dayofweek)
#     ddf = ddf.assign(pickup_hour = ddf.pickup_datetime.dt.hour)
    
#     ddf = ddf.drop(columns=['on_scene_datetime', 'request_datetime',
#                             'pickup_datetime', 'dispatching_base_num',
#                             'originating_base_num', 'shared_request_flag',
#                            'shared_match_flag' 'dropoff_datetime',
#                            ]
#                   )

#     ddf = ddf.dropna(how="any")
#     ddf = ddf.repartition(partition_size="128MB")
#     ddf = ddf.reset_index(drop=True)

#     categories = ["pickup_month", "pickup_dow", "pickup_hour",
#                  "dropoff_month", "dropoff_dow", "dropoff_hour",
#                  "hvfhs_license_num",]
#     for cat in categories:
#         ddf[cat] = ddf[cat].astype('category')
#     ddf = ddf.categorize(columns=categories)

#     df = ddf.sample(frac=0.0025).compute()
#     df.to_parquet("data/rides.parquet")
# subset_data()
# client.close()
# cluster.shutdown()

## Here we start the EDA portion of our pipeline with Pandas

### Let's grab some information about taxi zones / boroughs

In [ ]:
taxi_df = pd.read_csv("data/taxi+_zone_lookup.csv", usecols=["LocationID", "Borough"])
taxi_df.head()

In [ ]:
# These are the unique Boroughs in the taxi zone lookup table
taxi_df.Borough.unique().tolist()

The taxi_df includes two `LocationID` values that correspond to `Unknown` Boroughs.  
We need to know if these exist in the dataset

In [ ]:
taxi_df.loc[taxi_df['Borough'] == "Unknown"]

## And take a look at the distribution of travel times and the presence of `Unknown` boroughs

In [ ]:
df = pd.read_parquet("data/rides.parquet").reset_index(drop=True)
df.head()

In [ ]:
len(df.index) // 1e6

In [ ]:
df.memory_usage(deep=True).sum() / 2**20

We can see from below that over 9000 entries exist where travel occurs in or out of a borough that 
is an `Unknown` zone.

In [ ]:
df.loc[df.PULocationID.isin([263, 264])]

## We can see from the below that, as expected, our travel times are not normally distributed

In [ ]:
df['trip_time'].hist(bins=100)

In [ ]:
df['trip_time'].min()

In [ ]:
df['trip_time'].max()

We need to filter outliers from our data.  For this exercise, we will use `1.5 * Interquartile Range`
We can find the bounds using `df.quartile(method="median_unbiased")`.

In [ ]:
Q1 = df['trip_time'].quantile(0.25, interpolation="median_unbiased")
Q3 = df['trip_time'].quantile(0.75, interpolation="median_unbiased")
print(f"Q1:  {Q1}, Q3:  {Q3}")

As a check -- when porting code from Pandas to Dask, it was discovered that the Dask quantiles method does not
exactly replicate the Pandas / Numpy method.  See [here](https://docs.dask.org/en/stable/generated/dask.dataframe.DataFrame.quantile.html) and 
[here](https://github.com/dask/dask/issues/6566) for details.  Let's compare the results of the Dask and Pandas implementations 

In [ ]:
print(f"Using the `dask` method:  {dd.from_pandas(df['trip_time'], npartitions=8).quantile(0.75).compute()}")

Let's plot the upper and lower bounds, then filter the data.

In [ ]:
lower_bound = Q1 - (1.5*(Q3 - Q1))
lower_bound = lower_bound if lower_bound > 0 else 0

upper_bound = Q3 + (1.5*(Q3 - Q1))
print(f"Lower bound is:  {lower_bound}")
print(f"Upper bound is:  {upper_bound}")

In [ ]:
df['trip_time'].hist(bins=100)
plt.axvline(lower_bound, color="r")
plt.axvline(upper_bound, color="r")
plt.title("Histogram of Trip Durations")
plt.xlabel("Trip Duration")
plt.savefig("data/trip_histogram.png")

In [ ]:
print(f"Fraction of data lost after filtering outliers:  {len(df.loc[df.trip_time > upper_bound].index) / len(df.index)}")

In [ ]:
size_raw_data = len(df.index)

In [ ]:
df = df.loc[(df.trip_time >= lower_bound) & (df.trip_time <= upper_bound)]

In [ ]:
print(f"Fraction of data remaining after removing outliers:  {len(df.index) / size_raw_data}")

In [ ]:
df.head()

## What are the mean trip times by DoW like?

In [ ]:
df.groupby('pickup_dow')['trip_time'].agg(["mean", "median"])

In [ ]:
pivoted = pd.pivot_table(df, values='trip_time', index="pickup_dow", columns="pickup_hour")
pivoted

#### What does the data look like?  

In [ ]:
print(f"Shorted and longest average trip times based on pickup hour and day of week are:  {pivoted.min().min()} and {pivoted.max().max()} respectively")

In [ ]:
sns.heatmap(pivoted)

### Let's build two tables of mean travel times, based on pickup_dow and pickup_hour to create continuous features

In [ ]:
pickup_hour_means = df.groupby("pickup_hour")["trip_time"].mean().to_frame().reset_index()
pickup_hour_means = pickup_hour_means.rename(columns={"trip_time": "mean_trip_time_by_pickup_hour"})
pickup_hour_means

In [ ]:
pickup_dow_means = df.groupby("pickup_dow")["trip_time"].mean().to_frame().reset_index()
pickup_dow_means = pickup_dow_means.rename(columns={"trip_time": "mean_trip_time_by_pickup_dow"})
pickup_dow_means

In [ ]:
print(len(df.index))
print(len(df.columns))

In [ ]:
df = pd.merge(df, pickup_hour_means, left_on="pickup_hour", right_on="pickup_hour")
df = pd.merge(df, pickup_dow_means, left_on="pickup_dow", right_on="pickup_dow")
print(len(df.index))
print(len(df.columns))

In [ ]:
df.head()

## Taking inspiration from [this paper on using geospatial characteristics to predict trip durations](https://amr4i.github.io/pdfs/nyc_taxi_times.pdf),   

We can assign each pickup and dropoff location to a `Superborough`, as defined by the following table.  To do this, we will  
add the PU and DO borough to each trip, then assign a Superborough.  We do this to determine if a trip occurs between 
Superboroughs

| Superborough    | Boroughs                 |
|-----------------| -------------------------|
| Superborough 1  | Manhattan, Bronx, & EWR  |
| Superborough 2  | Brooklyn & Queens        |
| Superborough 3  | Staten Island            |
| Unknown         | Unknown

In [ ]:
df = pd.merge(df, taxi_df, left_on="PULocationID", right_on="LocationID", how="inner")
df = df.rename(columns={"Borough": "PUBorough"})
df = df.drop(columns="LocationID")
df.head()

In [ ]:
df = pd.merge(df, taxi_df, left_on="DOLocationID", right_on="LocationID", how="inner")
df = df.rename(columns={"Borough": "DOBorough"})
df = df.drop(columns="LocationID")
df.head()

In [ ]:
df.shape

In [ ]:
borough_mapping = {
    "Manhattan": "Superborough 1",
    "Bronx": "Superborough 1",
    "EWR": "Superborough 1",
    "Brooklyn": "Superborough 2",
    "Queens": "Superborough 2",
    "Staten Island": "Superborough 3",
    "Unknown": "Unknown",
}

In [ ]:
PUSuperborough = [borough_mapping.get(x) for x in df.PUBorough.tolist()]
DOSuperborough = [borough_mapping.get(x) for x in df.DOBorough.tolist()]
cross_superborough = ["N" if i==j else "Y" for (i,j) in zip(PUSuperborough, DOSuperborough)]

In [ ]:
PUSuperborough_DOSuperborough_Pair = [f"{i}-{j}" for i,j in zip(PUSuperborough, DOSuperborough)]

In [ ]:
df = df.assign(CrossSuperborough = cross_superborough)

In [ ]:
df = df.assign(PUSuperborough_DOSuperborough = PUSuperborough_DOSuperborough_Pair)

In [ ]:
df.head()

In [ ]:
c = df.columns.tolist()
c

In [ ]:
df['congestion_surcharge'].hist(bins=100)

In [ ]:
df['airport_fee'].hist(bins=100)

In [ ]:
df['tolls'].value_counts()

In [ ]:
df['congestion_surcharge'].hist(bins=100)

In [ ]:
df['CrossSuperborough'].value_counts()

In [ ]:
df.dtypes

## Final Cleanup

In [ ]:
df['airport_fee'] = df['airport_fee'].replace("None", 0)
df['airport_fee'] = df['airport_fee'].replace('nan', 0)
df['airport_fee'] = df['airport_fee'].astype(float)

In [ ]:
df['airport_fee'] = df['airport_fee'].fillna(0)

In [ ]:
df.hvfhs_license_num.value_counts()

In [ ]:
df.wav_request_flag.value_counts()

In [ ]:
df.wav_match_flag.value_counts()

In [ ]:
df.CrossSuperborough.value_counts()

In [ ]:
df.access_a_ride_flag.value_counts()

In [ ]:
df.PUBorough.value_counts().plot(kind='bar')

In [ ]:
df.DOBorough.value_counts().plot(kind='bar')

In [ ]:
to_drop = ['base_passenger_fare', 'dropoff_month', 'dropoff_dow', 'dropoff_hour',
           'bcf', 'sales_tax', 'tips', 'driver_pay', 'dropoff_datetime', 'access_a_ride_flag', 'wav_match_flag'
          ]

In [ ]:
df2 = df.drop(columns=to_drop)

In [ ]:
df2.head()

In [ ]:
categories = ['hvfhs_license_num', 'PULocationID', "DOLocationID", 'wav_request_flag', 'accessible_vehicle', 'pickup_month',
              'pickup_dow', 'pickup_hour', 'PUBorough', 'DOBorough', 'CrossSuperborough', 'PUSuperborough_DOSuperborough']

In [ ]:
df2[categories] = df2[categories].astype('category')

In [ ]:
df2['PUSuperborough_DOSuperborough'].unique()

## Train a Scikit Learn Dummy Regressor and evaluate model performance

In [ ]:
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
df2.dtypes

In [ ]:
df_valid = df2.sample(frac=0.1)

In [ ]:
df2.head()

In [ ]:
df2.columns.tolist()

In [ ]:
y = df2['trip_time']
X = df2.drop(columns='trip_time')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
mdl = DummyRegressor(strategy="mean")

In [ ]:
mdl.fit(X_train, y_train)

In [ ]:
y_predict = mdl.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, y_predict, squared=False)

In [ ]:
mse

## Linear Regression

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector as selector
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression

In [ ]:
categorical_transformer = OneHotEncoder(handle_unknown="ignore")
numeric_transformer = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, selector(dtype_exclude="category")),
        ("cat", categorical_transformer, selector(dtype_include="category")),
    ]
)
mdl = Pipeline(
    steps=[("preprocessor", preprocessor),
           ("regressor", LinearRegression()),
          ]
)
mdl.fit(X_train, y_train)

In [ ]:
mse = mean_squared_error(y_test, mdl.predict(X_test), squared=False)

In [ ]:
mse

## XGBoost

In [ ]:
import xgboost as xgb

In [ ]:
mdl2 = xgb.XGBRegressor(enable_categorical=True, tree_method="hist")
mdl2.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], )

In [ ]:
mse = mean_squared_error(y_test, mdl2.predict(X_test), squared=False)

In [ ]:
mse

In [ ]:
mdl2.get_xgb_params()

In [ ]:
mdl2.feature_importances_

In [ ]:
mdl2.feature_names_in_

In [ ]:
plt.bar(x=mdl2.feature_names_in_, height=mdl2.feature_importances_)
plt.xticks(rotation=90)